<a href="https://colab.research.google.com/github/mounikamarreddy/ML-Practice/blob/main/LGBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score

Dataset can be downloaded from the below link:
https://archive.ics.uci.edu/ml/datasets/Breast+Cancer+Wisconsin+(Diagnostic)


In [4]:
from google.colab import files
uploaded = files.upload()

Saving data.csv to data.csv


In [9]:
data = pd.read_csv("data.csv")

In [10]:
data = data.drop(['Unnamed: 32'],axis = 1)

In [15]:
data.head()

,id,Label,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [14]:
#Rename Dataset to Label to make it easy to understand
data = data.rename(columns={'diagnosis':'Label'})

In [18]:
#two class classification problem
data['Label'].value_counts()

B    357
M    212
Name: Label, dtype: int64

In [20]:
y = data["Label"].values

In [21]:
# Encoding categorical data
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
Y = labelencoder.fit_transform(y) # M=1 and B=0

In [23]:
#Define the independent variables. Let's also drop Gender, so we can normalize other data
X = data.drop(labels = ["Label", "id"], axis=1) 
feature_names = np.array(X.columns)  

In [24]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X)
X = scaler.transform(X)

In [25]:
##Split data into train and test to verify accuracy after fitting the model. 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [26]:
!pip install lightgbm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [27]:
import lightgbm as lgb
d_train = lgb.Dataset(X_train, label=y_train)

In [28]:
# https://lightgbm.readthedocs.io/en/latest/Parameters.html -documentation for different available parameters in LGBM
lgbm_params = {'learning_rate':0.05, 'boosting_type':'gbdt',    #Try dart for better accuracy
              'objective':'binary',
              'metric':['auc', 'binary_logloss'],
              'num_leaves':100,
              'max_depth':10}


In [29]:
clf = lgb.train(lgbm_params, d_train, 50) #50 iterations. Increase iterations for small learning rates
#Prediction on test data
y_pred_lgbm=clf.predict(X_test)

In [32]:
#convert into binary values 0/1 for classification
for i in range(0, X_test.shape[0]):
    if y_pred_lgbm[i]>=.5:       # setting threshold to .5
       y_pred_lgbm[i]=1
    else:  
       y_pred_lgbm[i]=0

In [33]:
cm_lgbm = confusion_matrix(y_test, y_pred_lgbm)

In [34]:
cm_lgbm

array([[70,  1],
       [ 4, 39]])

In [36]:
from sklearn import metrics
print ("Accuracy with LGBM = ", metrics.accuracy_score(y_pred_lgbm,y_test))

Accuracy with LGBM =  0.956140350877193


In [37]:
print("AUC score with LGBM is: ", roc_auc_score(y_pred_lgbm,y_test))

AUC score with LGBM is:  0.9604729729729731
